In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:

# #file saving location
images_dir = '/content/gdrive/My Drive/Colab Notebooks/Classes/IA 651/Audio-data-classification/Spectro_imgs/'

In [ ]:
import os
import glob
from PIL import Image

All images are of Width: 100 Height: 128

In [ ]:
for digit in range(1):
    # Directory of wav files
    folder_path = f"{images_dir}{digit}/"
    file_list = glob.glob(os.path.join(folder_path, '*'))

    for i, img_path  in enumerate(file_list):
    #   # Open the image file
      image = Image.open(f"{img_path}")

    #   # Get the size of the image
      width, height = image.size

      print(f"Width: {width} Height: {height}")

              

Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128
Width: 100 Height: 128


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:

img_width, img_height = 100, 128
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    images_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    images_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

test_generator = test_datagen.flow_from_directory(
    images_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 195 images belonging to 10 classes.
Found 45 images belonging to 10 classes.
Found 240 images belonging to 10 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
epochs = 50
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n//train_generator.batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.n//validation_generator.batch_size)


Epoch 1/50
6/6 [==============================] - 111s 21s/step - loss: 2.3135 - accuracy: 0.1166 - val_loss: 2.3056 - val_accuracy: 0.0625
Epoch 2/50
6/6 [==============================] - 7s 1s/step - loss: 2.3036 - accuracy: 0.1043 - val_loss: 2.3027 - val_accuracy: 0.1250
Epoch 3/50
6/6 [==============================] - 6s 936ms/step - loss: 2.2955 - accuracy: 0.1350 - val_loss: 2.2866 - val_accuracy: 0.1562
Epoch 4/50
6/6 [==============================] - 8s 1s/step - loss: 2.3210 - accuracy: 0.1288 - val_loss: 2.3057 - val_accuracy: 0.1250
Epoch 5/50
6/6 [==============================] - 6s 883ms/step - loss: 2.2865 - accuracy: 0.1288 - val_loss: 2.2674 - val_accuracy: 0.1562
Epoch 6/50
6/6 [==============================] - 8s 2s/step - loss: 2.2883 - accuracy: 0.1350 - val_loss: 2.2746 - val_accuracy: 0.1562
Epoch 7/50
6/6 [==============================] - 7s 1s/step - loss: 2.2878 - accuracy: 0.1288 - val_loss: 2.2440 - val_accuracy: 0.1250
Epoch 8/50
6/6 [================

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.n//test_generator.batch_size)
print('Test accuracy:', test_acc)


7/7 [==============================] - 3s 446ms/step - loss: 1.5678 - accuracy: 0.4554
Test accuracy: 0.4553571343421936


In [ ]:
import numpy as np

validation_generator.reset()  # reset the generator to the beginning of the validation dataset

predictions = model.predict(validation_generator, steps=validation_generator.n//validation_generator.batch_size+1)

predicted_classes = np.argmax(predictions, axis=1)

true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

print('Predictions:', predicted_classes)
print('True classes:', true_classes)

# Compute the confusion matrix
confusion_matrix = tf.math.confusion_matrix(labels=true_classes, predictions=predicted_classes)

# Print the confusion matrix
print('Confusion matrix:')
print(confusion_matrix)



2/2 [==============================] - 1s 111ms/step
Predictions: [0 0 6 0 1 3 6 7 4 6 1 9 3 7 3 7 4 5 9 8 3 5 5 7 9 8 9 0 6 2 7 5 9 1 0 0 3
 1 9 4 6 9 8 0 5]
True classes: [0 0 0 0 1 1 1 1 1 1 2 2 2 3 3 3 3 4 4 4 4 4 5 5 5 5 6 6 6 6 6 7 7 7 7 8 8
 8 8 8 9 9 9 9 9]
Confusion matrix:
tf.Tensor(
[[3 0 0 0 0 0 1 0 0 0]
 [0 1 0 1 1 0 2 1 0 0]
 [0 1 0 1 0 0 0 0 0 1]
 [0 0 0 1 1 0 0 2 0 0]
 [0 0 0 1 0 2 0 0 1 1]
 [0 0 0 0 0 1 0 1 1 1]
 [1 0 1 0 0 0 1 1 0 1]
 [1 1 0 0 0 1 0 0 0 1]
 [1 1 0 1 1 0 0 0 0 1]
 [1 0 0 0 0 1 1 0 1 1]], shape=(10, 10), dtype=int32)
